In [1]:
import requests
import re
import pandas as pd
import xml.etree.ElementTree as ET
import json
import os
import sys
import matplotlib.pyplot as plt
from collections import Counter
import xml.etree.ElementTree as ET
from typing import List, Dict, Any, Optional
import duckdb

In [2]:
lexeme_df = pd.read_parquet("../data/ruland-dictionaries.parquet")
lexeme_df["target_len"] = lexeme_df["target_canonical"].str.split().str.len()

In [3]:
lexeme_df["target_len"].value_counts()

target_len
1    1803
2     854
3     195
4      74
5      28
6      14
7       8
9       1
Name: count, dtype: int64

In [19]:
MAX_N = 3
lexeme_df = lexeme_df[lexeme_df["target_len"] <= MAX_N].copy()
len(lexeme_df)

2852

In [5]:
conn = duckdb.connect('/srv/data/grela/grela_v0-4.duckdb', read_only=True)

In [6]:
# 128 GB RAM → give DuckDB plenty, but leave headroom for Python/OS/file cache
conn.execute("""
  SET memory_limit = '96GB';              -- or '80GB' if you run multiple jobs
  SET threads = 8;                        -- raise gradually (16/24/32) if stable
  SET preserve_insertion_order = false;
  PRAGMA temp_directory='/srv/data/duckdb_tmp';   -- fast SSD/NVMe
  PRAGMA max_temp_directory_size='2TB';          -- whatever your disk allows
""")

In [7]:
# choose a fast, large temp dir (adjust path)
os.makedirs("/srv/data/duckdb_tmp", exist_ok=True)
conn.execute("""
  PRAGMA temp_directory='/srv/data/duckdb_tmp';
  PRAGMA max_temp_directory_size='500GB';   -- fit your disk
""")

In [8]:
# Build a global order for ALL EMLAP tokens (with punctuation, for pretty KWIC)
conn.execute("""
CREATE OR REPLACE TEMP VIEW emlap_all_tokens AS
SELECT
  t.grela_id,
  t.sentence_id,
  s.position        AS sentence_position,
  t.token_id,
  t.token_text,
  LOWER(t.lemma)    AS lemma_lower,
  t.pos,
  t.char_start,
  t.char_end,
  ROW_NUMBER() OVER (
    PARTITION BY t.grela_id
    ORDER BY s.position, t.char_start
  ) AS global_token_pos
FROM tokens t
JOIN works w  ON t.grela_id = w.grela_id
JOIN sentences s USING (sentence_id)
WHERE w.grela_id LIKE 'emlap%';
""")

# A content-only stream for matching (no punctuation, lemma required), BUT keep the global pos
conn.execute("""
CREATE OR REPLACE TEMP VIEW emlap_content_tokens AS
SELECT
  a.*,
  ROW_NUMBER() OVER (
    PARTITION BY a.grela_id
    ORDER BY a.sentence_position, a.char_start
  ) AS global_content_pos
FROM emlap_all_tokens a
WHERE a.lemma_lower IS NOT NULL
  AND a.pos <> 'PUNCT';
""")

conn.execute("""
-- Full stream with a stable per-work order
CREATE OR REPLACE TEMP TABLE emlap_full_stream AS
SELECT
  t.grela_id,
  t.sentence_id,
  s.position AS sentence_position,
  t.token_id,
  t.token_text,
  LOWER(t.lemma) AS lemma_lower,
  t.pos,
  t.char_start,
  t.char_end,
  ROW_NUMBER() OVER (
    PARTITION BY t.grela_id
    ORDER BY s.position, t.char_start
  ) AS seq_full
FROM tokens t
JOIN works w  ON t.grela_id = w.grela_id
JOIN sentences s USING (sentence_id)
WHERE w.grela_id LIKE 'emlap%';

-- Content-only stream aligned to full stream with precomputed next hops/grams
CREATE OR REPLACE TEMP TABLE emlap_content_stream AS
WITH c AS (
  SELECT
    f.*,
    ROW_NUMBER() OVER (
      PARTITION BY f.grela_id
      ORDER BY f.sentence_position, f.char_start
    ) AS seq_content
  FROM emlap_full_stream f
  WHERE f.lemma_lower IS NOT NULL AND f.pos <> 'PUNCT'
)
SELECT
  c.*,
  LEAD(c.lemma_lower, 1) OVER (PARTITION BY c.grela_id ORDER BY c.seq_content) AS l2,
  LEAD(c.lemma_lower, 2) OVER (PARTITION BY c.grela_id ORDER BY c.seq_content) AS l3,
  LEAD(c.seq_full,     1) OVER (PARTITION BY c.grela_id ORDER BY c.seq_content) AS next1_seq_full,
  LEAD(c.seq_full,     2) OVER (PARTITION BY c.grela_id ORDER BY c.seq_content) AS next2_seq_full,
  -- optional prejoined strings to avoid CONCAT in the hot path
  CASE WHEN LEAD(c.lemma_lower,1) OVER (PARTITION BY c.grela_id ORDER BY c.seq_content) IS NOT NULL
       THEN c.lemma_lower || ' ' || LEAD(c.lemma_lower,1) OVER (PARTITION BY c.grela_id ORDER BY c.seq_content)
  END AS n2,
  CASE WHEN LEAD(c.lemma_lower,2) OVER (PARTITION BY c.grela_id ORDER BY c.seq_content) IS NOT NULL
       THEN c.lemma_lower || ' ' || LEAD(c.lemma_lower,1) OVER (PARTITION BY c.grela_id ORDER BY c.seq_content) || ' ' ||
            LEAD(c.lemma_lower,2) OVER (PARTITION BY c.grela_id ORDER BY c.seq_content)
  END AS n3
FROM c;
""")

In [16]:
import re
import unicodedata
import pandas as pd
from typing import Optional

def concordance_for_target_across_sentences(
    conn,
    target_canonical: str | None,
    target_relemmatized: str | None,
    window: int = 10,
    out_path: Optional[str] = None,   # <-- NEW: stream directly to Parquet if set
):
    """
    Cross-sentence KWIC in EMLAP with strict adjacency, searching BOTH lemma and token_text
    for BOTH target_canonical and target_relemmatized.

    If out_path is provided, writes Parquet via DuckDB COPY (no pandas memory) and returns None.
    Otherwise, returns a pandas DataFrame (may be memory-heavy).
    """

    # ---------- Normalization helpers ----------
    def _strip_diacritics(s: str) -> str:
        return "".join(ch for ch in unicodedata.normalize("NFKD", s) if not unicodedata.combining(ch))

    def _latin_norm(s: str | None) -> str | None:
        if not s or not isinstance(s, str):
            return None
        s = s.strip().lower()
        s = _strip_diacritics(s)
        s = (s.replace("æ", "ae").replace("œ", "oe").replace("j", "i").replace("v", "u"))
        s = s.replace("_", " ")
        s = re.sub(r"\s+", " ", s).strip()
        return s or None

    def _prep(t: str | None):
        if not t:
            return 0, ("", "", ""), ""
        words = t.split()
        if not (1 <= len(words) <= 4):
            raise ValueError("Only 1–3-word targets supported (MAX_N=4).")
        w = tuple(words + ["", "", ""])[:3]
        return len(words), w, " ".join(words)

    tc = _latin_norm(target_canonical)
    tr = _latin_norm(target_relemmatized)
    if not tc and not tr:
        raise ValueError("Provide at least one of target_canonical or target_relemmatized.")

    tc_len, (tc_w1, tc_w2, tc_w3), tc_phrase = _prep(tc)
    tr_len, (tr_w1, tr_w2, tr_w3), tr_phrase = _prep(tr)

    # ---------- SQL-side normalizer ----------
    def NORM(expr: str) -> str:
        return ("replace(replace(replace(replace(lower({x}), 'æ', 'ae'), 'œ', 'oe'), 'j', 'i'), 'v', 'u')"
                .format(x=expr))

    LEMMA_OR_TOKEN_NORM = NORM("coalesce(nullif(cs.lemma_lower, ''), cs.token_text)")

    # NOTE: two CTEs (target_sentence_texts / tokens) now limited to sentences actually needed
    sql = f"""
WITH raw_matches AS (
  SELECT cs.grela_id, cs.sentence_id AS target_sentence_id, cs.seq_full AS start_seq_full,
         ?::INT AS target_len, 'lemma' AS matched_by, 'canonical' AS target_from, ?::VARCHAR AS target_phrase
  FROM emlap_content_stream cs
  WHERE ? AND (
    (? = 1 AND {LEMMA_OR_TOKEN_NORM} = ?)
    OR (? = 2 AND {NORM('cs.n2')} = ?)
    OR (? = 3 AND {NORM('cs.n3')} = ?)
  )

  UNION ALL
  SELECT cs.grela_id, cs.sentence_id, cs.seq_full,
         ?::INT, 'lemma', 'relemmatized', ?::VARCHAR
  FROM emlap_content_stream cs
  WHERE ? AND (
    (? = 1 AND {LEMMA_OR_TOKEN_NORM} = ?)
    OR (? = 2 AND {NORM('cs.n2')} = ?)
    OR (? = 3 AND {NORM('cs.n3')} = ?)
  )

  UNION ALL
  SELECT f1.grela_id, f1.sentence_id, f1.seq_full,
         ?::INT, 'token', 'canonical', ?::VARCHAR
  FROM emlap_full_stream f1
  LEFT JOIN emlap_full_stream f2
    ON f2.grela_id = f1.grela_id AND f2.seq_full = f1.seq_full + 1
  LEFT JOIN emlap_full_stream f3
    ON f3.grela_id = f1.grela_id AND f3.seq_full = f1.seq_full + 2
  WHERE ? AND (
    (? = 1 AND {NORM('f1.token_text')} = ?)
    OR (? = 2 AND {NORM('f1.token_text')} = ? AND {NORM('f2.token_text')} = ?)
    OR (? = 3 AND {NORM('f1.token_text')} = ? AND {NORM('f2.token_text')} = ? AND {NORM('f3.token_text')} = ?)
  )

  UNION ALL
  SELECT f1.grela_id, f1.sentence_id, f1.seq_full,
         ?::INT, 'token', 'relemmatized', ?::VARCHAR
  FROM emlap_full_stream f1
  LEFT JOIN emlap_full_stream f2
    ON f2.grela_id = f1.grela_id AND f2.seq_full = f1.seq_full + 1
  LEFT JOIN emlap_full_stream f3
    ON f3.grela_id = f1.grela_id AND f3.seq_full = f1.seq_full + 2
  WHERE ? AND (
    (? = 1 AND {NORM('f1.token_text')} = ?)
    OR (? = 2 AND {NORM('f1.token_text')} = ? AND {NORM('f2.token_text')} = ?)
    OR (? = 3 AND {NORM('f1.token_text')} = ? AND {NORM('f2.token_text')} = ? AND {NORM('f3.token_text')} = ?)
  )
),
ranked AS (
  SELECT *, ROW_NUMBER() OVER (
    PARTITION BY grela_id, target_sentence_id, start_seq_full
    ORDER BY
      CASE matched_by WHEN 'lemma' THEN 0 ELSE 1 END,
      CASE target_from WHEN 'canonical' THEN 0 ELSE 1 END
  ) AS rn
  FROM raw_matches
),
uniq_matches AS (
  SELECT grela_id, target_sentence_id, start_seq_full, target_len, matched_by, target_from, target_phrase
  FROM ranked
  WHERE rn = 1
),
bounds AS (
  SELECT m.grela_id, m.target_sentence_id, m.start_seq_full, m.target_len, m.matched_by, m.target_from, m.target_phrase,
         CASE m.target_len WHEN 1 THEN m.start_seq_full WHEN 2 THEN cs.next1_seq_full WHEN 3 THEN cs.next2_seq_full END AS end_seq_full
  FROM uniq_matches m
  JOIN emlap_content_stream cs
    ON cs.grela_id = m.grela_id AND cs.seq_full = m.start_seq_full
  WHERE CASE m.target_len
          WHEN 1 THEN TRUE
          WHEN 2 THEN cs.next1_seq_full = m.start_seq_full + 1
          WHEN 3 THEN cs.next2_seq_full = m.start_seq_full + 2
        END
),
context AS (
  SELECT b.grela_id, b.target_sentence_id, b.start_seq_full, b.target_len, b.matched_by, b.target_from, b.target_phrase,
         f.sentence_id, f.token_id, f.token_text, f.lemma_lower, f.pos, f.char_start, f.char_end,
         ROW_NUMBER() OVER (
           PARTITION BY b.grela_id, b.target_sentence_id, b.start_seq_full
           ORDER BY f.seq_full
         ) AS ord
  FROM bounds b
  JOIN emlap_full_stream f
    ON f.grela_id = b.grela_id
   AND f.seq_full BETWEEN (b.start_seq_full - ?) AND (b.end_seq_full + ?)
),
agg_kwic AS (
  SELECT
    grela_id, target_sentence_id, start_seq_full,
    ANY_VALUE(target_len) AS target_len,
    ANY_VALUE(matched_by) AS matched_by,
    ANY_VALUE(target_from) AS target_from,
    ANY_VALUE(target_phrase) AS target_phrase,
    LIST(sentence_id ORDER BY ord) AS window_sentence_ids,
    STRING_AGG(token_text, ' ' ORDER BY ord) AS kwic_text,
    LIST(
      STRUCT_PACK(
        token_id := token_id,
        token_text := token_text,
        lemma := lemma_lower,
        pos := pos,
        sentence_id := sentence_id,
        char_start := char_start,
        char_end := char_end
      )
      ORDER BY ord
    ) AS kwic_tokens
  FROM context
  GROUP BY grela_id, target_sentence_id, start_seq_full
),

-- ---- NEW: limit sentence-level aggregations to the sentences we actually need ----
needed_sentences AS (
  SELECT DISTINCT grela_id, target_sentence_id AS sentence_id
  FROM agg_kwic
),

target_sentence_texts AS (
  SELECT e.grela_id, e.sentence_id,
         STRING_AGG(e.token_text, ' ' ORDER BY e.char_start) AS sentence_text
  FROM emlap_full_stream e
  JOIN needed_sentences n
    ON n.grela_id = e.grela_id AND n.sentence_id = e.sentence_id
  GROUP BY e.grela_id, e.sentence_id
),

target_sentence_tokens AS (
  SELECT e.grela_id, e.sentence_id,
         LIST(
           STRUCT_PACK(
             token_id := e.token_id,
             token_text := e.token_text,
             lemma := e.lemma_lower,
             pos := e.pos,
             char_start := e.char_start,
             char_end := e.char_end
           )
           ORDER BY e.char_start
         ) AS sentence_tokens
  FROM emlap_full_stream e
  JOIN needed_sentences n
    ON n.grela_id = e.grela_id AND n.sentence_id = e.sentence_id
  GROUP BY e.grela_id, e.sentence_id
)

SELECT
  a.target_phrase,
  a.target_from,
  a.matched_by,
  a.grela_id,
  a.target_sentence_id,
  a.window_sentence_ids[1]  AS start_sentence_id,
  a.window_sentence_ids[-1] AS end_sentence_id,
  a.kwic_text,
  a.kwic_tokens,
  tst.sentence_text      AS target_sentence_text,
  tstok.sentence_tokens  AS target_sentence_tokens
FROM agg_kwic a
LEFT JOIN target_sentence_texts  tst
  ON tst.grela_id = a.grela_id AND tst.sentence_id = a.target_sentence_id
LEFT JOIN target_sentence_tokens tstok
  ON tstok.grela_id = a.grela_id AND tstok.sentence_id = a.target_sentence_id
ORDER BY a.grela_id, a.target_sentence_id, a.start_seq_full;
"""

    params = [
        # 1) lemma canonical
        tc_len, tc_phrase or "", bool(tc),
        tc_len, (tc or ""), tc_len, (tc or ""), tc_len, (tc or ""),
        # 2) lemma relemmatized
        tr_len, tr_phrase or "", bool(tr),
        tr_len, (tr or ""), tr_len, (tr or ""), tr_len, (tr or ""),
        # 3) token canonical
        tc_len, tc_phrase or "", bool(tc),
        tc_len, tc_w1,
        tc_len, tc_w1, tc_w2,
        tc_len, tc_w1, tc_w2, tc_w3,
        # 4) token relemmatized
        tr_len, tr_phrase or "", bool(tr),
        tr_len, tr_w1,
        tr_len, tr_w1, tr_w2,
        tr_len, tr_w1, tr_w2, tr_w3,
        # window
        window, window,
    ]

    if out_path:
        # 1) Strip trailing semicolon from inner SELECT
        sql_nosemi = sql.rstrip().rstrip(';')

        # 2) Safely embed the path as a SQL string literal (COPY TO doesn't accept param here)
        out_quoted = "'" + out_path.replace("'", "''") + "'"

        # 3) Execute COPY with a literal filename
        conn.execute(f"COPY ({sql_nosemi}) TO {out_quoted} (FORMAT PARQUET);", params)
        return None
    else:
        return conn.execute(sql, params).fetch_df()

In [8]:
lexeme_df[lexeme_df["target_len"] > 1].head(5)

,Lemma,target_canonical,entry_ids,entry_count,types,variants,notes,senses_flat,sense_count,sense_def_all,sense_def_strict_all,sense_raw_all,translations_all,target_relemmatized,target_len
23,Lapis maior,lapis maior,"[Ruland1612-Lapis-maior, Ruland1612-Lapis-maio...",3,[L],[],[],"[{'def': None, 'entry_id': 'Ruland1612-Lapis-m...",3,"[finde nach etlicher Meynung, die calcinirten ...","[None, None, id est, der Geist außgezogen auß ...","[finde nach etlicher Meynung, die calcinirten ...","[{'entry_id': 'Ruland1612-Lapis-maior', 'quote...",lapis magnus,2
24,Leo viridis,leo viridis,"[Ruland1612-Leo-viridis, Ruland1612-Leo-viridi...",3,[L],[],[],"[{'def': None, 'entry_id': 'Ruland1612-Leo-vir...",3,"[ist Hermetis Erz Glaß ond vitriol, vnd das Bl...","[None, quorundam opinione aurum, vitriolum]","[ist Hermetis Erz Glaß ond vitriol, vnd das Bl...","[{'entry_id': 'Ruland1612-Leo-viridis', 'quote...",leo uiridis,2
25,Machina tractoria,machina tractoria,"[Ruland1612-Machina-tractoria, Ruland1612-Mach...",3,[M],[],[],"[{'def': None, 'entry_id': 'Ruland1612-Machina...",3,"[Haspel, Gezeugsso Derg ond Waͤsser heben., Sc...","[None, None, None]","[Haspel, Gezeugsso Derg ond Waͤsser heben., Sc...","[{'entry_id': 'Ruland1612-Machina-tractoria', ...",machina tractoria,2
30,Panum aereorum species,panum aereorum species,"[Ruland1612-Panum-aereorum-species-,-Vorpleits...",3,[P],[],[],"[{'def': None, 'entry_id': 'Ruland1612-Panum-a...",3,"[Vorpleitstein, Vorpleitloch., Lech, zwir Vorp...","[None, None, None]","[Vorpleitstein, Vorpleitloch., Lech, zwir Vorp...",[{'entry_id': 'Ruland1612-Panum-aereorum-speci...,panis aereus species,3
32,Pyritae aurei coloris,pyritae aurei coloris,[Ruland1612-Pyritae-aurei-coloris-.-1.-Pyrites...,3,[P],[],[],"[{'def': '1', 'entry_id': 'Ruland1612-Pyritae-...",3,"[1, è quibus ignis elicitur, è quibus ignis no...","[1, è quibus ignis elicitur, è quibus ignis no...",[1. Pyrites qui est solidus & colorem auri tot...,[{'entry_id': 'Ruland1612-Pyritae-aurei-colori...,pyritus aureus color,3


In [9]:
lexeme_df[lexeme_df["target_canonical"].str.startswith("mercurius")]

,Lemma,target_canonical,entry_ids,entry_count,types,variants,notes,senses_flat,sense_count,sense_def_all,sense_def_strict_all,sense_raw_all,translations_all,target_relemmatized,target_len
27,Mercurius,mercurius,"[Ruland1612-Mercurius, Ruland1612-Mercurius, R...",3,[M],[],[],"[{'def': 'id est, sulphur.', 'entry_id': 'Rula...",3,"[id est, sulphur., est principium materiale, v...","[id est, sulphur., est principium materiale, v...","[id est, sulphur. Mercurius. Mercurius ist in ...","[{'entry_id': 'Ruland1612-Mercurius', 'quote':...",mercurius,1
96,Mercurius metallorum,mercurius metallorum,"[Ruland1612-Mercurius-metallorum, Ruland1612-M...",2,[M],[],[],[{'def': 'Ist darauß die Natur der Coͤrper gez...,2,[Ist darauß die Natur der Coͤrper gezogen wird...,[Ist darauß die Natur der Coͤrper gezogen wird...,[Ist darauß die Natur der Coͤrper gezogen wird...,[{'entry_id': 'Ruland1612-Mercurius-metallorum...,mercurius metallum,2
1967,Mercurius a natura coagulatus,mercurius a natura coagulatus,[Ruland1612-Mercurius-a-natura-coagulatus-est-...,1,[M],[],[],[{'def': 'est solidum metallum. Arte vero coag...,1,"[est solidum metallum. Arte vero coagulatur, a...","[est solidum metallum. Arte vero coagulatur, a...","[est solidum metallum. Arte vero coagulatur, a...",[],mercurius ab natura coagulo,4
1968,Mercurius argentipigmentum,mercurius argentipigmentum,[Ruland1612-Mercurius-argentipigmentum],1,[M],[],[],"[{'def': 'ist Schwefel victriol, Alaun Saltz-d...",1,"[ist Schwefel victriol, Alaun Saltz-dieweils d...","[ist Schwefel victriol, Alaun Saltz-dieweils d...","[ist Schwefel victriol, Alaun Saltz-dieweils d...",[{'entry_id': 'Ruland1612-Mercurius-argentipig...,mercurius argentipigmentum,2
1969,Mercurius chambar,mercurius chambar,[Ruland1612-Mercurius-chambar],1,[M],[],[],"[{'def': 'ist magnesia, fuessend Coͤrper und W...",1,"[ist magnesia, fuessend Coͤrper und Wasser.]","[ist magnesia, fuessend Coͤrper und Wasser.]","[ist magnesia, fuessend Coͤrper und Wasser.]","[{'entry_id': 'Ruland1612-Mercurius-chambar', ...",mercurius chaambar,2
1970,Mercurius corallinus,mercurius corallinus,[Ruland1612-Mercurius-corallinus],1,[M],[],[],[{'def': 'qui per oleum ouorum & aquas alias i...,1,[qui per oleum ouorum & aquas alias in rubedin...,[qui per oleum ouorum & aquas alias in rubedin...,[qui per oleum ouorum & aquas alias in rubedin...,[],mercurius corallinus,2
1971,Mercurius crudus,mercurius crudus,[Ruland1612-Mercurius-crudus],1,[M],[],[],"[{'def': 'est is, qui nondum separatus est a s...",1,"[est is, qui nondum separatus est a sua matric...","[est is, qui nondum separatus est a sua matric...","[est is, qui nondum separatus est a sua matric...","[{'entry_id': 'Ruland1612-Mercurius-crudus', '...",mercurius crudus,2
1972,Mercurius crystallinus,mercurius crystallinus,"[Ruland1612-Mercurius-crystallinus,]",1,[M],[],[],[{'def': 'qui saepe sublimatus est in formam c...,1,[qui saepe sublimatus est in formam crystalli ...,[qui saepe sublimatus est in formam crystalli ...,[qui saepe sublimatus est in formam crystalli ...,[],mercurius crystallinus,2
1973,Mercurius laxus,mercurius laxus,[Ruland1612-Mercurius-laxus],1,[M],[],[],"[{'def': 'est turbith minerale.', 'entry_id': ...",1,[est turbith minerale.],[est turbith minerale.],[est turbith minerale.],[],mercurius laxus,2
1974,Mercurius mineralium,mercurius mineralium,[Ruland1612-Mercurius-mineralium],1,[M],[],[],"[{'def': 'est oleitas, vel vnctuositas de mine...",1,"[est oleitas, vel vnctuositas de mineris auri ...","[est oleitas, vel vnctuositas de mineris auri ...","[est oleitas, vel vnctuositas de mineris auri ...",[{'entry_id': 'Ruland1612-Mercurius-mineralium...,mercurius mineralis,2


In [10]:
target_canonical     = "lapis philosophorum"
target_relemmatized  = "lapis philosophus"

df = concordance_for_target_across_sentences(
    conn, target_canonical, target_relemmatized, window=10
)

In [12]:
df.head(10)

,target_phrase,target_from,matched_by,grela_id,target_sentence_id,start_sentence_id,end_sentence_id,kwic_text,kwic_tokens,target_sentence_text,target_sentence_tokens
0,lapis philosophus,relemmatized,lemma,emlap_100002,emlap_100002_156,emlap_100002_156,emlap_100002_156,", aquarum tam uegetabilium quam mineralium ad ...","[{'token_id': 1383892, 'token_text': ',', 'lem...","Et cum hoc sufficit tibi modus generalis , aqu...","[{'token_id': 1383885, 'token_text': 'Et', 'le..."
1,lapis philosophus,relemmatized,lemma,emlap_100002,emlap_100002_672,emlap_100002_671,emlap_100002_672,quia ibi large continetur hec doctrina . PItem...,"[{'token_id': 1397417, 'token_text': 'quia', '...",PItem in compositione lapidis philosophorum ru...,"[{'token_id': 1397424, 'token_text': 'PItem', ..."
2,lapis philosophus,relemmatized,lemma,emlap_100002,emlap_100002_721,emlap_100002_719,emlap_100002_722,"& uirtutem sue nature . PQuestio . Pater , si ...","[{'token_id': 1398678, 'token_text': '&', 'lem...","Pater , si lapis philosophorum cadit in doctri...","[{'token_id': 1398685, 'token_text': 'Pater', ..."
3,lapis philosophus,relemmatized,lemma,emlap_100002,emlap_100002_722,emlap_100002_722,emlap_100002_722,"non solum ad preciosos lapides , sed et iam ad...","[{'token_id': 1398700, 'token_text': 'non', 'l...","Fili , hec regula non solum ad preciosos lapid...","[{'token_id': 1398696, 'token_text': 'Fili', '..."
4,lapis philosophus,relemmatized,lemma,emlap_100007,emlap_100007_356,emlap_100007_355,emlap_100007_357,operemur nisi in sole et Mercurio simuliunctis...,"[{'token_id': 2081304, 'token_text': 'operemur...",in quibus lapis philosophorum consistit .,"[{'token_id': 2081312, 'token_text': 'in', 'le..."
5,lapis philosophus,relemmatized,lemma,emlap_100007,emlap_100007_538,emlap_100007_537,emlap_100007_538,"emendat naturam , nisi in suam naturam . Tres ...","[{'token_id': 2084237, 'token_text': 'emendat'...","Tres sunt lapides Philosophorum principales , ...","[{'token_id': 2084245, 'token_text': 'Tres', '..."
6,lapis philosophus,relemmatized,lemma,emlap_100007,emlap_100007_588,emlap_100007_587,emlap_100007_588,", ut multi putant , sed de corpore extractus ....","[{'token_id': 2085167, 'token_text': ',', 'lem...","Lapis Philosophorum ex tribus , scilicet sole ...","[{'token_id': 2085177, 'token_text': 'Lapis', ..."
7,lapis philosophus,relemmatized,lemma,emlap_100007,emlap_100007_64,emlap_100007_63,emlap_100007_64,", ut est ostensum . Item , Sal metallorum est ...","[{'token_id': 2075648, 'token_text': ',', 'lem...","Item , Sal metallorum est lapis Philosophorum ...","[{'token_id': 2075653, 'token_text': 'Item', '..."
8,lapis philosophus,relemmatized,lemma,emlap_100007,emlap_100007_676,emlap_100007_676,emlap_100007_677,", & quadrangulo triangulum , fac circulum rotu...","[{'token_id': 2086760, 'token_text': ',', 'lem...","Fac de masculo & foemina circulum rotundum , &...","[{'token_id': 2086747, 'token_text': 'Fac', 'l..."
9,lapis philosophus,relemmatized,lemma,emlap_100007,emlap_100007_789,emlap_100007_788,emlap_100007_790,super Epistolam Hermetis ait . Solus ille qui ...,"[{'token_id': 2088471, 'token_text': 'super', ...",Solus ille qui scit facere lapidem philosophor...,"[{'token_id': 2088476, 'token_text': 'Solus', ..."


In [11]:
lexeme_df_sample_emlap_instances = lexeme_df.sample(10, random_state=1).apply(lambda row: concordance_for_target_across_sentences(conn, row["target_canonical"], row["target_relemmatized"], window=10).to_dict("records"), axis=1)
lexeme_df_sample_emlap_instances

1128    [{'target_phrase': 'cutis', 'target_from': 'ca...
192                                                    []
2228                                                   []
45      [{'target_phrase': 'asseres', 'target_from': '...
2517                                                   []
682                                                    []
2812                                                   []
1662                                                   []
2269    [{'target_phrase': 'purgo argentum', 'target_f...
1534    [{'target_phrase': 'glutinum tenacitas', 'targ...
dtype: object

In [32]:
from pathlib import Path
import re
import pandas as pd
from pathlib import Path

outdir = Path("../data/large_files/emlap_ruland_instances/")
outdir.mkdir(exist_ok=True)

In [23]:

def safe_name(s: str | None) -> str:
    """Make a safe short filename component from a target phrase."""
    if not s or not isinstance(s, str):
        return "unknown"
    s = s.strip().lower().replace(" ", "_")
    s = re.sub(r"[^a-z0-9_]+", "", s)   # keep only safe chars
    return s or "unnamed"


for i, row in lexeme_df.iterrows():
    if i > 1600:
        tc = row.get("target_canonical")
        tr = row.get("target_relemmatized")
        target = tc or tr
        name = safe_name(target)
        fpath = str(outdir / f"{i:05d}_{name}.parquet")

            # streams directly to Parquet (no df = ..., no df.to_parquet)
        concordance_for_target_across_sentences(conn, tc, tr, window=10, out_path=fpath)
        print(f"[{i}] saved → {fpath}")


[1601] saved → emlap_hits/01601_hydrophorbia.parquet
[1602] saved → emlap_hits/01602_hydropiper.parquet
[1603] saved → emlap_hits/01603_hypoglossum.parquet
[1604] saved → emlap_hits/01604_iacens.parquet
[1605] saved → emlap_hits/01605_iassa.parquet
[1606] saved → emlap_hits/01606_icteritia_rubea.parquet
[1607] saved → emlap_hits/01607_idam.parquet
[1608] saved → emlap_hits/01608_idiotae.parquet
[1609] saved → emlap_hits/01609_idraogiros.parquet
[1610] saved → emlap_hits/01610_idroagira.parquet
[1611] saved → emlap_hits/01611_iffides.parquet
[1612] saved → emlap_hits/01612_ignis_algit.parquet
[1613] saved → emlap_hits/01613_ignis_calor.parquet
[1614] saved → emlap_hits/01614_ignis_clare_ardens.parquet
[1615] saved → emlap_hits/01615_ignis_elementaris.parquet
[1616] saved → emlap_hits/01616_ignis_exstinctus.parquet
[1617] saved → emlap_hits/01617_ignis_leonis.parquet
[1618] saved → emlap_hits/01618_ignis_persicus.parquet
[1619] saved → emlap_hits/01619_ignis_pruinus_adeptus.parquet
[1621

In [28]:
fnames = []
for i, row in lexeme_df.iterrows():
            tc = row.get("target_canonical")
            tr = row.get("target_relemmatized")
            target = tc or tr
            name = safe_name(tc or tr)
            fname = f"{i:05d}_{name}.parquet"
            #if fname not in in results_meta:
            #fpath = outdir / fname
            fnames.append(fname)
fnames[-10:]

['02965_anima.parquet',
 '02966_aqua_coelestis.parquet',
 '02967_arsaltos.parquet',
 '02969_cedue.parquet',
 '02970_fibula_ferrea.parquet',
 '02972_leux_amygdala.parquet',
 '02973_lignum_crucis.parquet',
 '02974_minera_vena_terrae.parquet',
 '02975_nebulgea.parquet',
 '02976_ranac_10.parquet']

In [29]:
lexeme_df.loc[2970]

Lemma                                                       Fibula ferrea
target_canonical                                            fibula ferrea
entry_ids                                      [Ruland1612-Fibula-ferrea]
entry_count                                                             1
types                                                                 [F]
variants                                                               []
notes                                                                  []
senses_flat                                                            []
sense_count                                                             0
sense_def_all                                                          []
sense_def_strict_all                                                   []
sense_raw_all                                                          []
translations_all        [{'entry_id': 'Ruland1612-Fibula-ferrea', 'quo...
target_relemmatized                   

In [33]:
pd.read_parquet((os.path.join(outdir, fnames[0]))).to_dict("records")[:10]

[{'target_phrase': 'naphtha',
  'target_from': 'canonical',
  'matched_by': 'lemma',
  'grela_id': 'emlap_100012',
  'target_sentence_id': 'emlap_100012_3225',
  'start_sentence_id': 'emlap_100012_3224',
  'end_sentence_id': 'emlap_100012_3225',
  'kwic_text': 'sulfure , argentouiuo : item in bituminum generibus diuersis , naphtha , succino , ambra , & petreleo , cui nomen',
  'kwic_tokens': array([{'token_id': 343356, 'token_text': 'sulfure', 'lemma': 'sulfur', 'pos': 'NOUN', 'sentence_id': 'emlap_100012_3224', 'char_start': 64, 'char_end': 71},
         {'token_id': 343357, 'token_text': ',', 'lemma': ',', 'pos': 'PUNCT', 'sentence_id': 'emlap_100012_3224', 'char_start': 71, 'char_end': 72},
         {'token_id': 343358, 'token_text': 'argentouiuo', 'lemma': 'argentouiuus', 'pos': 'ADJ', 'sentence_id': 'emlap_100012_3224', 'char_start': 73, 'char_end': 84},
         {'token_id': 343359, 'token_text': ':', 'lemma': ':', 'pos': 'PUNCT', 'sentence_id': 'emlap_100012_3224', 'char_start':

In [34]:
len(os.listdir(outdir))

2927

In [54]:
#lexeme_df["emlap_instances"] = lexeme_df.apply(lambda row: concordance_for_target_across_sentences(conn, row["target_canonical"], row["target_relemmatized"], window=10).to_dict("records"), axis=1)

In [35]:
lexeme_df["fname"] = fnames

In [36]:
def read_hits(fname):
    try:
        path = os.path.join(outdir, fname)
        instances = pd.read_parquet(path).to_dict("records")
    except:
        instances = []
    return instances
lexeme_df["emlap_instances"] = lexeme_df["fname"].apply(read_hits)

In [37]:
lexeme_df["emlap_instances_N"] = lexeme_df["emlap_instances"].apply(len)

In [38]:
lexeme_df.sort_values("emlap_instances_N", ascending=False)[:20]

,Lemma,target_canonical,entry_ids,entry_count,types,variants,notes,senses_flat,sense_count,sense_def_all,sense_def_strict_all,sense_raw_all,translations_all,target_relemmatized,target_len,fname,emlap_instances,emlap_instances_N
479,Aqua,aqua,[Ruland1612-Aqua],1,[A],[],[1. Dieses Wasser reiniget/maschet/meitet/mach...,"[{'def': 'id est, liquor', 'entry_id': 'Ruland...",1,"[id est, liquor]","[id est, liquor]","[, id est, liquor, das Wasser ist auch immer d...","[{'entry_id': 'Ruland1612-Aqua', 'quote': 'das...",aqua,1,00479_aqua.parquet,"[{'target_phrase': 'aqua', 'target_from': 'can...",20140
40,A,a,"[Ruland1612-A, Ruland1612-A-,-Ein-Diamanttaffe...",2,[A],[],[],"[{'def': 'vide ana', 'entry_id': 'Ruland1612-A...",2,"[vide ana, , Ein Diamanttaffel.]","[vide ana, None]","[vide ana, , Ein Diamanttaffel.]","[{'entry_id': 'Ruland1612-A-,-Ein-Diamanttaffe...",a,1,00040_a.parquet,"[{'target_phrase': 'a', 'target_from': 'canoni...",13138
1082,Corpus,corpus,[Ruland1612-Corpus],1,[C],[],[],"[{'def': 'Clang', 'entry_id': 'Ruland1612-Corp...",1,[Clang],[Clang],[Clang. Buce. Der Coͤrper ist ein metallisch W...,"[{'entry_id': 'Ruland1612-Corpus', 'quote': 'D...",corpus,1,01082_corpus.parquet,"[{'target_phrase': 'corpus', 'target_from': 'c...",12498
76,Ignis,ignis,"[Ruland1612-Ignis, Ruland1612-Ignis]",2,[I],[],[],"[{'def': None, 'entry_id': 'Ruland1612-Ignis',...",2,[Ist nach etlicher Meinung das Oel sokauff der...,"[None, Ignis pro lapide philos]",[Ist nach etlicher Meinung das Oel sokauff der...,"[{'entry_id': 'Ruland1612-Ignis', 'quote': 'Is...",ignis,1,00076_ignis.parquet,"[{'target_phrase': 'ignis', 'target_from': 'ca...",11656
617,Aurum,aurum,[Ruland1612-Aurum],1,[A],[],"[Natiuum purum, quod a natura tale est, cuius ...","[{'def': 'à Germanis', 'entry_id': 'Ruland1612...",1,[à Germanis],[à Germanis],"[, à Germanis Goldtsa Chymistis dicitur Sol, &...","[{'entry_id': 'Ruland1612-Aurum', 'quote': 'Go...",aurum,1,00617_aurum.parquet,"[{'target_phrase': 'aurum', 'target_from': 'ca...",7927
537,Argentum,argentum,"[Ruland1612-Argentum-à-Chymistis-Luna,-cui-eti...",1,[A],[Argentum],[],"[{'def': 'à Chymistis Luna, cui etiam tribuitu...",1,"[à Chymistis Luna, cui etiam tribuitur]","[à Chymistis Luna, cui etiam tribuitur]","[à Chymistis Luna, cui etiam tribuitur: estque...",[{'entry_id': 'Ruland1612-Argentum-à-Chymistis...,argentum,1,00537_argentum.parquet,"[{'target_phrase': 'argentum', 'target_from': ...",7391
29,Oleum,oleum,"[Ruland1612-Oleum, Ruland1612-Oleum, Ruland161...",3,[O],[],[],"[{'def': 'id est, ignis, wirdt außgezogen von ...",3,"[id est, ignis, wirdt außgezogen von einer tro...","[id est, ignis, wirdt außgezogen von einer tro...","[id est, ignis, wirdt außgezogen von einer tro...","[{'entry_id': 'Ruland1612-Oleum', 'quote': 'wi...",oleum,1,00029_oleum.parquet,"[{'target_phrase': 'oleum', 'target_from': 'ca...",7005
2610,Spiritus,spiritus,[Ruland1612-Spiritus],1,[S],[],[],"[{'def': 'est aqua soluens è re simplici, & ac...",1,"[est aqua soluens è re simplici, & acri produc...","[est aqua soluens è re simplici, & acri produc...","[est aqua soluens è re simplici, & acri produc...","[{'entry_id': 'Ruland1612-Spiritus', 'quote': ...",spiritus,1,02610_spiritus.parquet,"[{'target_phrase': 'spiritus', 'target_from': ...",6559
97,Metallum,metallum,"[Ruland1612-Metallum, Ruland1612-Metallum]",2,[M],[],[],"[{'def': 'vena, Ertz.', 'entry_id': 'Ruland161...",2,"[vena, Ertz., id est, conflatum argentum.]","[vena, Ertz., id est, conflatum argentum.]","[vena, Ertz., id est, conflatum argentum.]","[{'entry_id': 'Ruland1612-Metallum', 'quote': ...",metallum,1,00097_metallum.parquet,"[{'target_phrase': 'metallum', 'target_from': ...",5676
10,Sulphur,sulphur,"[Ruland1612-Sulphur, Ruland1612-Sulphur, Rulan...",4,[S],[],[],"[{'def': 'chibur, vel, Albusao Arabice dicitur...",4,"[chibur, vel, Albusao Arabice dicitur, pars la...","[chibur, vel, Albusao Arabice dicitur, pars la...","[chibur, vel, Albusao Arabice dicitur, pars la...","[{'entry_id': 'Ruland161

In [39]:
lexeme_df["instances_ids"] = lexeme_df["emlap_instances"].apply(lambda x: [ins["grela_id"][6:] for ins in x])

In [40]:
emlap_metadata = pd.read_csv(
        "https://raw.githubusercontent.com/CCS-ZCU/EMLAP_ETL/refs/heads/master/data/emlap_metadata.csv",
        sep=";",
    )
emlap_metadata.head(5)

,working_title,filenames,no.,is_done,is_noscemus,if_noscemus_id,AUTHORSHIP,is_one_author,#if more than 1 author skip section and choose compendium below,is_author_known,...,publisher_comments,CONTENTS,genre,subject,SOURCE OF FILE,link,source_of_file,origin_of_copy,other_notes,tokens_N
0,"Augurello, Chrysopoeia",100001_Augurello1515_Chrysopoeia_GB_Noscemus,100001,True,True,713324.0,NaN,True,NaN,True,...,NaN,NaN,didactic poem,alchemy,NaN,https://wiki.uibk.ac.at/noscemus/Chrysopoeia,GB,Noscemus,NaN,23718
1,"Pseudo-Lull, Secretis",100002_Pseudo-Lull1518_De secretis_naturae_MDZ...,100002,True,False,NaN,NaN,True,NaN,True,...,NaN,NaN,treatise,"alchemy, medicine",NaN,https://www.digitale-sammlungen.de/en/view/bsb...,MDZ,MBS,NaN,24673
2,"Pantheus, Ars Transmutatione",100003_Pantheus1518_Ars_Transmutationis_Metall...,100003,True,False,NaN,NaN,True,NaN,True,...,NaN,NaN,treatise,alchemy,NaN,https://www.google.co.uk/books/edition/Ars_Tra...,GB,BL,NaN,8646
3,"Anon, Vera alchemiae",100004_Anon1561_Verae_Alchemiae_MDZ_MBS,100004,True,False,NaN,NaN,True,NaN,True,...,NaN,NaN,"compendium, florilegium",alchemy,NaN,https://mdz-nbn-resolving.de/details:bsb10141168,MDZ,MBS,NaN,3521
4,"Pantheus, Voarchadumia",100005_Pantheus1530_Voarchadumia_ONB,100005,True,False,NaN,NaN,True,NaN,True,...,NaN,NaN,treatise,alchemy,NaN,https://data.onb.ac.at/rep/10588E49,ONB,ONB,NaN,20386


In [41]:
emlap_id_title_dict = dict(zip(emlap_metadata['no.'].astype(str), emlap_metadata['working_title']))
emlap_id_date_dict = dict(zip(emlap_metadata['no.'].astype(str), emlap_metadata['date_publication']))

In [42]:
def add_emlap_metadata(instances_dict_list):
    instances_dict_list_new = []
    for instance_dict in instances_dict_list:
        id = instance_dict["grela_id"][6:]
        instance_dict["title"] = emlap_id_title_dict[id]
        instance_dict["year"] = emlap_id_date_dict[id]
        instances_dict_list_new.append(instance_dict)
    return instances_dict_list_new

In [43]:
instances_enriched = lexeme_df["emlap_instances"].apply(add_emlap_metadata)

In [44]:
lexeme_df["emlap_instances"] = instances_enriched

In [ ]:
lexeme_df.to_json("../data/large_files/ruland-emlap-grela.json")

In [ ]:
lexeme_df.to_parquet("../data/large_files/ruland-emlap-grela.parquet")


In [90]:
lexeme_df.to_json("../data/large_files/ruland-emlap.json")